In [1]:
#Load selinimum and automatically install the Chrome Driver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import re
from selenium.webdriver.chrome.service import Service

For versions of Chrome beyond 114, the WebDriver no longer has the capability to automatically retrieve a compatible version. In response, I manually acquired the WebDriver version 116 from the following website: https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/116.0.5845.96/win64/chromedriver-win64.zip. You can find additional information regarding this download on this page: https://support.google.com/chrome/thread/230521170/requires-version-116-of-the-google-chrome-driver%EF%BC%8Cplease?hl=en.

In [2]:
#hrome_driver_path = r'D:\classScraper\chromedriver-win64/chromedriver.exe'

service = Service(executable_path='D:\classScraper\chromedriver-win64/chromedriver.exe')
options = webdriver.ChromeOptions()
# running in optional headless mode
#options.add_argument('--headless')
driver = webdriver.Chrome(service=service, options=options)
time.sleep(10)

If you have chrome version older than 115, you can use this to automatically find the compatible webdriver.

In [3]:
"""#Setup Chrome driver 
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)
time.sleep(10)"""

"#Setup Chrome driver \noptions = Options()\noptions.add_argument('--headless')\ndriver = webdriver.Chrome(options=options)\ntime.sleep(10)"

In [4]:
from bs4 import BeautifulSoup

url = "https://student.msu.edu/psc/public/EMPLOYEE/SA/c/NUI_FRAMEWORK.PT_AGSTARTPAGE_NUI.GBL?CONTEXTIDPARAMS=TEMPLATE_ID%3aPTPPNAVCOL&scname=MSU_AA_SCHEDULE_NEW0&PanelCollapsible=Y"
driver.get(url)
time.sleep(20)

The ids might change. I am using this currently.

In [5]:
Semester = {'Summer 2020': "'SSR_CSTRMPRV_VW_DESCR$8'",
            'Fall 2020': "'SSR_CSTRMPRV_VW_DESCR$span$7'",
            'Spring 21': "'SSR_CSTRMPRV_VW_DESCR$span$6'",
            'Summer 21': "'SSR_CSTRMPRV_VW_DESCR$5'",
            'Fall 21': "'SSR_CSTRMPRV_VW_DESCR$4'",
            'Spring 22': "'SSR_CSTRMPRV_VW_DESCR$span$3'",
            'Summer 22': "'SSR_CSTRMPRV_VW_DESCR$2'",
            'Fall 22': "'SSR_CSTRMPRV_VW_DESCR$1'",
            'Spring 23': "'SSR_CSTRMPRV_VW_DESCR$0'",
            'Summer 23': "'SSR_CSTRMCUR_VW_DESCR$0'", 
            'Fall 23': "'SSR_CSTRMCUR_VW_DESCR$1'",
           'Spring 24': "'SSR_CSTRMCUR_VW_DESCR$2'",
           'Summer 24': "'SSR_CSTRMCUR_VW_DESCR$3'"}
Semester = {
    'Current': "'SSR_CSTRMCUR_VW_DESCR$0'",
    'next': "'SSR_CSTRMCUR_VW_DESCR$1'"
}

            
url = f"javascript:submitAction_win0(document.win0,{Semester['Current']});"

In [6]:
driver.execute_script(url);
time.sleep(30)

In [7]:
from selenium.webdriver.common.by import By
element = driver.find_element(By.ID, 'MSU_CLSRCH_WRK2_SUBJECT')  
element.send_keys("CMSE") #pick cmse for example

In [8]:
url = f"javascript:submitAction_win0(document.win0,'MSU_CLSRCH_WRK_SSR_PB_SEARCH');"
driver.execute_script(url);
time.sleep(20)

Function to get basic classes' info

In [9]:
def get_class_info(soup):
    values = []
    divs = soup.find_all("div", class_="ps-htmlarea")
    for div in divs:
        val = (div.get_text(strip=True))
        values.append(val)
    values.pop(0)
    del values[0]
    for i in range(5,len(values),10):
        values[i] = 'delete'
        values[i+1] = 'delete'
        values[i+2] = 'delete'
        values[i+3] = 'delete'
    values = list(filter(lambda x: x != "delete", values))   
    reshaped_list = [values[i:i+6] for i in range(0, len(values), 6)]
    # Create a DataFrame from the reshaped list
    col_names = ['Course', 'Type', 'Section', 'Schedule', 'Dates', 'Instructor']
    df = pd.DataFrame(reshaped_list, columns=col_names)
    df[['Course Code', 'Course Name']] = df['Course'].str.split(':', 1, expand=True)
    #df[['Type', 'Units']] = df['Type'].str.split('(', 1, expand=True)
    split_result = df['Type'].str.split('(', 1, expand=True)

    # Check if the split operation resulted in two columns
    if len(split_result.columns) == 2:
        df[['Type', 'Units']] = split_result
    else:
        # Handle the case where the split didn't result in two columns
        df['Type'] = split_result[0]  # Assign the first part to 'Type'
        df['Units'] = '' 
    df[['Section', 'Class Nbr', 'Academic Session']] = df['Section'].str.split('/', 2, expand=True)
    df[['Days', 'Time']] = df['Schedule'].str.split(':', 1, expand=True)
    df[['Units','Status']] = df['Units'].str.split(')',1,expand=True)
    df[['Subject','Course Number']] = df['Course Code'].str.split(' ',1,expand=True)

    df = df.drop(['Course', 'Schedule','Course Code','Instructor'], axis=1)
    df = df[['Subject','Course Number','Course Name','Type','Units','Status','Section','Class Nbr','Academic Session','Days','Time','Dates']]
    df['Units'] = df['Units'].str.extract(r'(\d+(?:\.\d+)?)')
    df['Section'] = df['Section'].str.extract(r'(\d+(?:\.\d+)?)')
    df['Class Nbr'] = df['Class Nbr'].str.extract(r'(\d+(?:\.\d+)?)')
    return df

In [10]:
def get_advanced_info(soup):
    loc = None
    email = None
    name = None
    values = []
    divs = soup.find_all("div", class_="ps-htmlarea")
    for div in divs:
        val = (div.get_text(strip=True))
        values.append(val)
    values = list(filter(lambda x: x != "", values))
    if len(values) > 5 and values[5] != "":
        string = values[5].split('Instructor:')
    else:
        return None, None, None
    loc = string[0]
    a_elements = soup.find_all('a')
    for a_element in a_elements:
        # Check if the 'href' attribute exists
        if 'href' in a_element.attrs:
            # Check if the href attribute contains "mailto:"
            if 'mailto:' in a_element['href']:
                # Extract the email address from the href attribute
                email = a_element['href'].split(':')[1]
                name = a_element.text
                break
    return loc, email, name

In [11]:
def add_info(df):
    location = []
    emails = []
    names = []
    for i in range(len(df)):
        element = driver.find_element(By.ID, f"DESCR100$0_row_{i}") 
        element.click()
        time.sleep(2)
        driver.switch_to.frame(0)
        body = driver.page_source
        soup = BeautifulSoup(body, 'html.parser')
        loc, email, name = get_advanced_info(soup)
        location.append(loc)
        emails.append(email)
        names.append(name)
        cancel_cmd="javascript:doUpdateParent(document.win0,'#ICCancel');"
        driver.execute_script(cancel_cmd);
        time.sleep(2)
    df['Location'] = location
    df['email'] = emails
    df['Instructor'] = names
    return df

In [12]:
body = driver.page_source
soup = BeautifulSoup(body, 'html.parser')
df = get_class_info(soup)  # getting info on the first page

In [13]:
# get the number of results
result_element = soup.find('span', id='MSU_RSLT_NAV_WK_PTPG_ROWS_GRID')
# Extract the text content
result_text = result_element.get_text(strip=True)
result = int(result_text.split()[0])
pages = (result + 49) // 50# get the number of aggregated pages
pages

1

In [14]:
df = add_info(df)

In [15]:
for i in range(1,pages):
    button = driver.find_element(By.ID, 'MSU_RSLT_NAV_WK_SEARCH_CONDITION2')
    button.click()
    time.sleep(2)
    body = driver.page_source
    soup = BeautifulSoup(body, 'html.parser')
    df_new = get_class_info(soup)
    df_new = add_info(df_new)
    df = df.append(df_new, ignore_index=True)
df

,Subject,Course Number,Course Name,Type,Units,Status,Section,Class Nbr,Academic Session,Days,Time,Dates,Location,email,Instructor
0,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Open,001,14272,Regular Academic Session,Mon Wed,8:00 AM-9:50 AM,8/28/2023 - 12/10/2023,STEM 3201,wangj164@msu.edu,Jianrong Wang
1,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Open,002,14273,Regular Academic Session,Mon Wed,10:20 AM-12:10 PM,8/28/2023 - 12/10/2023,STEM 3201,mengsen@msu.edu,Mengsen Zhang
2,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Wait List,003,14274,Regular Academic Session,Mon Wed,12:40 PM-2:30 PM,8/28/2023 - 12/10/2023,STEM 3201,baolianz@msu.edu,Lianzhang Bao
3,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Wait List,004,14275,Regular Academic Session,Mon Wed,3:00 PM-4:50 PM,8/28/2023 - 12/10/2023,STEM 3201,chitwoo9@msu.edu,Daniel Chitwood
4,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Open,005,14276,Regular Academic Session,Tue Thu,8:00 AM-9:50 AM,8/28/2023 - 12/10/2023,STEM 3201,baolianz@msu.edu,Lianzhang Bao
5,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Open,006,14277,Regular Academic Session,Tue Thu,10:20 AM-12:10 PM,8/28/2023 - 12/10/2023,STEM 3201,salmonra@msu.edu,Rachel Frisbie
6,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Open,007,14278,Regular Academic Session,Tue Thu,12:40 PM-2:30 PM,8/28/2023 - 12/10/2023,STEM 3201,rmlarose@msu.edu,Ryan LaRose
7,CMSE,202,Computational Modeling and Data Analysis II,Lecture,4.00,Wait List,001,14279,Regular Academic Session,Mon Wed,10:20 AM-12:10 PM,8/28/2023 - 12/10/2023,Wonders Hall C211,dsilvia@msu.edu,Devin Silvia
8,CMSE,202,Computational Modeling and Data Analysis II,Lecture,4.00,Open,002,14280,Regular Academic Session,Mon Wed,3:00 PM-4:50 PM,8/28/2023 - 12/10/2023,Chemistry 323,karniksa@msu.edu,Santhosh Karnik
9,CMSE,202,Computational Modeling and Data Analysis II,Lecture,4.00,Open,003,14281,Regular Academic Session,Tue Thu,3:00 PM-4:50 PM,8/28/2023 - 12/10/2023,STEM 3201,aalessio@msu.edu,Adam Alessio


In [16]:
def replace_empty_with_none(value):
    return None if pd.isna(value) or (isinstance(value, str) and value == '') else value

In [17]:
df['Approval Required'] = df['Dates'].apply(lambda x: 'Yes' if 'Approval Required' in x else 'No')
df['Dates'] = df['Dates'].str.replace('Approval Required', '').str.strip()
# Create a new column "Approval Required" with "Yes" for rows where "Dates" originally contained "Approval Required," and "No" otherwise

df[['first_name','last_name']] = df['Instructor'].str.split(' ',n=1,expand=True)
df

,Subject,Course Number,Course Name,Type,Units,Status,Section,Class Nbr,Academic Session,Days,Time,Dates,Location,email,Instructor,Approval Required,first_name,last_name
0,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Open,001,14272,Regular Academic Session,Mon Wed,8:00 AM-9:50 AM,8/28/2023 - 12/10/2023,STEM 3201,wangj164@msu.edu,Jianrong Wang,No,Jianrong,Wang
1,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Open,002,14273,Regular Academic Session,Mon Wed,10:20 AM-12:10 PM,8/28/2023 - 12/10/2023,STEM 3201,mengsen@msu.edu,Mengsen Zhang,No,Mengsen,Zhang
2,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Wait List,003,14274,Regular Academic Session,Mon Wed,12:40 PM-2:30 PM,8/28/2023 - 12/10/2023,STEM 3201,baolianz@msu.edu,Lianzhang Bao,No,Lianzhang,Bao
3,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Wait List,004,14275,Regular Academic Session,Mon Wed,3:00 PM-4:50 PM,8/28/2023 - 12/10/2023,STEM 3201,chitwoo9@msu.edu,Daniel Chitwood,No,Daniel,Chitwood
4,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Open,005,14276,Regular Academic Session,Tue Thu,8:00 AM-9:50 AM,8/28/2023 - 12/10/2023,STEM 3201,baolianz@msu.edu,Lianzhang Bao,No,Lianzhang,Bao
5,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Open,006,14277,Regular Academic Session,Tue Thu,10:20 AM-12:10 PM,8/28/2023 - 12/10/2023,STEM 3201,salmonra@msu.edu,Rachel Frisbie,No,Rachel,Frisbie
6,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Open,007,14278,Regular Academic Session,Tue Thu,12:40 PM-2:30 PM,8/28/2023 - 12/10/2023,STEM 3201,rmlarose@msu.edu,Ryan LaRose,No,Ryan,LaRose
7,CMSE,202,Computational Modeling and Data Analysis II,Lecture,4.00,Wait List,001,14279,Regular Academic Session,Mon Wed,10:20 AM-12:10 PM,8/28/2023 - 12/10/2023,Wonders Hall C211,dsilvia@msu.edu,Devin Silvia,No,Devin,Silvia
8,CMSE,202,Computational Modeling and Data Analysis II,Lecture,4.00,Open,002,14280,Regular Academic Session,Mon Wed,3:00 PM-4:50 PM,8/28/2023 - 12/10/2023,Chemistry 323,karniksa@msu.edu,Santhosh Karnik,No,Santhosh,Karnik
9,CMSE,202,Computational Modeling and Data Analysis II,Lecture,4.00,Open,003,14281,Regular Academic Session,Tue Thu,3:00 PM-4:50 PM,8/28/2023 - 12/10/2023,STEM 3201,aalessio@msu.edu,Adam Alessio,No,Adam,Alessio


In [24]:
df['net_id'] = df['email'].str.split('@').str[0]
df

,Subject,Course Number,Course Name,Type,Units,Status,Section,Class Nbr,Academic Session,Days,Time,Dates,Location,email,Instructor,Approval Required,first_name,last_name,net_id
0,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Open,001,14272,Regular Academic Session,Mon Wed,8:00 AM-9:50 AM,8/28/2023 - 12/10/2023,STEM 3201,wangj164@msu.edu,Jianrong Wang,No,Jianrong,Wang,wangj164
1,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Open,002,14273,Regular Academic Session,Mon Wed,10:20 AM-12:10 PM,8/28/2023 - 12/10/2023,STEM 3201,mengsen@msu.edu,Mengsen Zhang,No,Mengsen,Zhang,mengsen
2,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Wait List,003,14274,Regular Academic Session,Mon Wed,12:40 PM-2:30 PM,8/28/2023 - 12/10/2023,STEM 3201,baolianz@msu.edu,Lianzhang Bao,No,Lianzhang,Bao,baolianz
3,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Wait List,004,14275,Regular Academic Session,Mon Wed,3:00 PM-4:50 PM,8/28/2023 - 12/10/2023,STEM 3201,chitwoo9@msu.edu,Daniel Chitwood,No,Daniel,Chitwood,chitwoo9
4,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Open,005,14276,Regular Academic Session,Tue Thu,8:00 AM-9:50 AM,8/28/2023 - 12/10/2023,STEM 3201,baolianz@msu.edu,Lianzhang Bao,No,Lianzhang,Bao,baolianz
5,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Open,006,14277,Regular Academic Session,Tue Thu,10:20 AM-12:10 PM,8/28/2023 - 12/10/2023,STEM 3201,salmonra@msu.edu,Rachel Frisbie,No,Rachel,Frisbie,salmonra
6,CMSE,201,Computational Modeling and Data Analysis I,Lecture,4.00,Open,007,14278,Regular Academic Session,Tue Thu,12:40 PM-2:30 PM,8/28/2023 - 12/10/2023,STEM 3201,rmlarose@msu.edu,Ryan LaRose,No,Ryan,LaRose,rmlarose
7,CMSE,202,Computational Modeling and Data Analysis II,Lecture,4.00,Wait List,001,14279,Regular Academic Session,Mon Wed,10:20 AM-12:10 PM,8/28/2023 - 12/10/2023,Wonders Hall C211,dsilvia@msu.edu,Devin Silvia,No,Devin,Silvia,dsilvia
8,CMSE,202,Computational Modeling and Data Analysis II,Lecture,4.00,Open,002,14280,Regular Academic Session,Mon Wed,3:00 PM-4:50 PM,8/28/2023 - 12/10/2023,Chemistry 323,karniksa@msu.edu,Santhosh Karnik,No,Santhosh,Karnik,karniksa
9,CMSE,202,Computational Modeling and Data Analysis II,Lecture,4.00,Open,003,14281,Regular Academic Session,Tue Thu,3:00 PM-4:50 PM,8/28/2023 - 12/10/2023,STEM 3201,aalessio@msu.edu,Adam Alessio,No,Adam,Alessio,aalessio


In [25]:
df = df.applymap(replace_empty_with_none)

In [26]:
df.to_csv("Fall2023.csv",index=False)